<a href="https://colab.research.google.com/github/BusyApe201804930/assessment/blob/main/assessment1_Tianhao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# not sure if it is ok but using pytorch here instead of tensorflow and keras
%matplotlib inline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

In [ ]:
# Start of P1
train= pd.read_csv('/content/train-1.csv',header=None)
train.shape
train.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.443132,0.532405,0.437770,0.061966,0.783677,0.935789,0.909104,0.683616,0.895410,0.871122,0.733646
1,0.270151,0.563418,0.136042,0.164526,0.013159,0.731077,0.961079,0.202600,0.921334,0.731678,0.416784
2,0.571161,0.124119,0.826382,0.692914,0.817431,0.573339,0.248638,0.632073,0.606688,0.348984,0.590013
3,0.414595,0.541134,0.383968,0.696607,0.113355,0.591044,0.770690,0.016159,0.291561,0.666265,0.477864
4,0.069221,0.047482,0.269938,0.812681,0.533125,0.091919,0.581794,0.854649,0.298028,0.819031,0.415577


In [ ]:
test= pd.read_csv('/content/test-1.csv',header=None)
train.shape
test.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.381483,0.259656,0.828097,0.785679,0.951713,0.602431,0.631316,0.139366,0.182238,0.149903,0.491957
1,0.165943,0.447260,0.727400,0.639058,0.790171,0.038858,0.202324,0.098871,0.971737,0.992815,0.543159
2,0.555724,0.814016,0.142540,0.520290,0.569297,0.243047,0.396575,0.537876,0.741946,0.482729,0.529083
3,0.157568,0.989660,0.109685,0.849540,0.995821,0.954934,0.437646,0.688811,0.174563,0.808960,0.748886
4,0.283251,0.948084,0.997192,0.500534,0.214394,0.726381,0.548864,0.306034,0.989905,0.403511,0.524699


In [ ]:
length=int(len(train)*0.1)
length

10000

In [ ]:
valid=train.iloc[:10000,:]

In [ ]:
# setting the data set without the last column
x_train=train.iloc[:,:-1].values
y_train=train.iloc[:,-1].values
x_valid=valid.iloc[:,:-1].values
y_valid=valid.iloc[:,-1].values
x_test=test.iloc[:,:-1].values
y_test=test.iloc[:,-1].values

In [ ]:
x_train = torch.tensor(x_train, dtype = float)
y_train = torch.tensor(y_train, dtype = float)
x_valid = torch.tensor(x_valid , dtype = torch.float)
y_valid = torch.tensor(y_valid , dtype =torch.float)
x_test = torch.tensor(x_test, dtype = torch.float)
y_test = torch.tensor(y_test, dtype = torch.float)

In [ ]:
# specify single layer, batch 128 and 100 nods in the NN
# activation function using Relu here
# also define else for 1.4 use (sigmoid activation)
input_size =x_train.shape[1]
hidden_size = 128
output_size = 1
batch_size = 128
def my_model(lat_dim,input_size, output_size,activation='relu'):
    if activation=='relu':
        my_nn = torch.nn.Sequential(
            torch.nn.Linear(input_size, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, lat_dim),
            torch.nn.Linear(lat_dim, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, output_size),
        )
    else:
        my_nn = torch.nn.Sequential(
            torch.nn.Linear(input_size, 100),
            torch.nn.Sigmoid(),
            torch.nn.Linear(100, lat_dim),
            torch.nn.Linear(lat_dim, 100),
            torch.nn.Sigmoid(),
            torch.nn.Linear(100, output_size),
        )
    return my_nn


In [ ]:
# MSE loss with optimization SGD 
# test and validation loss here
lat_dim=5
def train(lat_dim,input_size, output_size,lr,epoch,batch_size,activation='relu',verbose=True):
    my_nn=my_model(lat_dim,input_size, output_size,activation='relu')
    cost = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(my_nn.parameters(), lr = lr)
    losses_valid = []
    losses_test = []
  

    for i in range(epoch):
        batch_loss_valid = []
        batch_loss_test = []
        for start in range(0, len(x_train), batch_size):
            end = start + batch_size if start + batch_size < len(x_train) else len(x_train)
            xx = torch.tensor(x_train[start:end], dtype = torch.float, requires_grad = True)
            yy = torch.tensor(y_train[start:end], dtype = torch.float, requires_grad = True)
            prediction = my_nn(xx)
            loss = cost(prediction, yy)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            

        prediction_valid = my_nn(x_valid)
        l_valid=cost(prediction_valid, y_valid)

        prediction_test = my_nn(x_train)
        l_train=cost(prediction_test, y_test)


        losses_valid.append(l_valid.data.numpy())
        losses_test.append(l_test.data.numpy())
        if verbose==True:
            print('valid loss for epoch '+str(i)+': '+str(l_valid.data.numpy()))
            print('test loss for epoch '+str(i)+': '+str(l_test.data.numpy()))
    return my_nn


In [ ]:
# P1.2 
# lat_dim = 5, epouch = 10
input_size = x_train.shape[1]
lat_dim = 5
output_size = 1
batch_size = 128
lr=1e-4
epoch=10
my_nn=train(lat_dim,input_size, output_size,lr,epoch,batch_size,activation='relu')

In [ ]:
# P1.2
# lat_dim = 100, epoch = 10
input_size =x_train.shape[1]
lat_dim = 100
output_size = 1
batch_size = 128
lr=1e-4
epoch=10
my_nn=train(lat_dim,input_size, output_size,lr,epoch,batch_size,activation='relu')

In [ ]:
# As can be seen from the results, 
# when lat_dim increases to 100, both valid loss and test loss increase

In [ ]:
# P1.3
# lat_dim = 100, epoch = 10
# lr with 0.01, 0.1 and 0.5
input_size =x_train.shape[1]
lat_dim = 100
output_size = 1
batch_size = 128
lr=[0.01,0.1,0.5]
epoch=10
mse_test=[]
for i in lr:
    my_nn=train(lat_dim,input_size, output_size,i,epoch,batch_size,activation='relu')
    prediction_test = my_nn(x_test)
    m=np.mean((prediction_test.data.numpy()-y_test.data.numpy())**2)
    mse_test.append(m)
mse_test


In [ ]:
# loss should decrease as lr increases

In [ ]:
# P1.4
# lat_dim = 100, epoch = 50
input_size =x_train.shape[1]
lat_dim = 100
output_size = 1
batch_size = 128
lr=1e-4
epoch=50
my_nn=train(lat_dim,input_size, output_size,lr,epoch,batch_size,activation='relu')

In [ ]:
# for sigmoid
input_size =x_train.shape[1]
lat_dim = 100
output_size = 1
batch_size = 128
lr=1e-4
epoch=50
my_nn=train(lat_dim,input_size, output_size,lr,epoch,batch_size,activation='sigmoid')

In [ ]:
# loss is better for relu, sigmoid performs worse in this case

In [ ]:
list(range(1,10))

In [ ]:
# P1.5
# Change lat_dim size with different lr, with epoch = 5, batch size = 128
lat_dim=list(range(1,10))+list(range(10,110,10))
output_size = 1
batch_size = 128
lr=[0.01,0.02,0.03,0.04,0.1]
epoch=5
mse_total=[]
for j in lat_dim:
    mse_valid=[]
    model=[]
    mse_test=[]
    for i in lr:
        my_nn=train(j,input_size, output_size,i,epoch,batch_size,activation='relu',verbose=False)
        prediction_valid = my_nn(x_valid)
        m=np.mean((prediction_valid.data.numpy()-y_valid.data.numpy())**2)
        mse_valid.append(m)
        prediction_test = my_nn(x_test)
        m=np.mean((prediction_test.data.numpy()-y_test.data.numpy())**2)
        mse_test.append(m)
        model.append(my_nn)
    mse_total.append(np.mean(mse_test))
    best_model=model[mse_valid.index(max(mse_valid))]
    prediction_test =best_model(x_test)
    m=np.mean((prediction_test.data.numpy()-y_test.data.numpy())**2)
    print(str(j)+': '+str(m))

In [ ]:
# plotting lat_dim against MSE here
# as lat_dim increases a little bit, mse decreases dramatically
# however as lat_dim grows, no big effect on mse, and when it is very large, mse tends to increase
plt.plot(lat_dim,mse_total)
plt.xlabel('lat_dim')
plt.ylabel('mse')

In [ ]:
# Start of P2
x_train= pd.read_csv('/content/xtrain-2.csv').drop('Unnamed: 0',axis=1)
y_train= pd.read_csv('/content/ytrain-2.csv').drop('Unnamed: 0',axis=1)
x_test= pd.read_csv('/content/xtest-2.csv').drop('Unnamed: 0',axis=1)
y_test= pd.read_csv('/content/ytest-2.csv').drop('Unnamed: 0',axis=1)

In [ ]:
y_train

In [ ]:
length=int(len(x_train)*0.1)
x_valid=x_train.iloc[:10000,:]
y_valid=y_train.iloc[:10000,:]

In [ ]:
x_train=x_train.values
y_train=y_train.values
x_valid=x_valid.values
y_valid=y_valid.values
x_test=x_test.values
y_test=y_test.values

In [ ]:
x_train = torch.tensor(x_train, dtype = float)
y_train = torch.tensor(y_train, dtype = float)
x_valid = torch.tensor(x_valid , dtype = torch.float)
y_valid = torch.tensor(y_valid , dtype =torch.float)
x_test = torch.tensor(x_test, dtype = torch.float)
y_test = torch.tensor(y_test, dtype = torch.float)

In [ ]:
# not very clear about defining the model, not working here
# ??
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
class my_model(torch.nn.Module):
    def __init__(self,input_size, output_size,l,w):

       

        self.linear_layers.append(torch.nn.ReLU())
#??
            

    
    def forward(self, x):
       
      
        return x



In [ ]:
# should be same as before but with Adam optimizer, not sure
def train(input_size, output_size,l,w,lr,epoch,batch_size,verbose=True):
    my_nn=my_model(input_size, output_size,l,w)
    cost = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(my_nn.parameters(), lr = lr,eps=1e-3)

    losses_valid = []
    losses_test = []

    for i in range(epoch):
        batch_loss_valid = []
        batch_loss_test = []

        for start in range(0, len(x_train), batch_size):
            end = start + batch_size if start + batch_size < len(x_train) else len(x_train)
            xx = torch.tensor(x_train[start:end], dtype = torch.float, requires_grad = True)
            yy = torch.tensor(y_train[start:end], dtype = torch.float, requires_grad = True)
            prediction = my_nn(xx)
            loss = cost(prediction, yy)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

        prediction_valid = my_nn(x_valid)
        l_valid=cost(prediction_valid, y_valid)


        prediction_test = my_nn(x_test)
        l_test=cost(prediction_test, y_test)

       
        losses_valid.append(l_valid.data.numpy())
        losses_test.append(l_test.data.numpy())
        if verbose==True:
            print('valid loss for epoch '+str(i)+': '+str(l_valid.data.numpy()))
            print('test loss for epoch '+str(i)+': '+str(l_test.data.numpy()))
    return my_nn

# following code for in different conditions should be similar, no result

In [ ]:
# 2 5
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=2
w=5
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# 2 10
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=2
w=10
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# 2 100
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=2
w=100
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# 2 200
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=2
w=200
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# 3 5
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=3
w=5
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# 3 10
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=3
w=10
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# 3 100
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=3
w=100
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# 3 200
input_size =x_train.shape[1]
output_size = 5
batch_size = 300
lr=1e-4
l=3
w=200
epoch=5
my_nn=train(input_size, output_size,l,w,lr,epoch,batch_size)

In [ ]:
# Didn't get the result but the intuition may be: as l and w increase, the loss would decrease